In [171]:
# path ./trainmusic/data/train/train_1092.mp3
#with open('./trainmusic/data/train/train_1092.mp3','rb') as f:
    #a = f.readline()

def zero_crossing_rate(wavedata, block_length, sample_rate):
    
    # how many blocks have to be processed?
    num_blocks = int(np.ceil(len(wavedata)/block_length))
    
    # when do these blocks begin (time in seconds)?
    #timestamps = (np.arange(0,num_blocks - 1) * (block_length / float(samplerate)))
    
    zcr = []
    
    for i in range(0,num_blocks-1):
        
        start = i * block_length
        stop  = np.min([(start + block_length - 1), len(wavedata)])
        
        zc = 0.5 * np.mean(np.abs(np.diff(np.sign(wavedata[start:stop]))))
        zcr.append(zc)
    
    return np.asarray(zcr)#, np.asarray(timestamps)

def root_mean_square(wavedata, block_length, sample_rate):
    
    # how many blocks have to be processed?
    num_blocks = int(np.ceil(len(wavedata)/block_length))
    
    # when do these blocks begin (time in seconds)?
    #timestamps = (np.arange(0,num_blocks - 1) * (block_length / float(samplerate)))
    
    rms = []
    
    for i in range(0,num_blocks-1):
        
        start = i * block_length
        stop  = np.min([(start + block_length - 1), len(wavedata)])
        
        rms_seg = np.mean(wavedata[start:stop]**2)
        rms.append(rms_seg)
    
    return np.asarray(rms)#, np.asarray(timestamps)


In [187]:
import audioread
import audioop
import numpy as np

winsize = 4410
total_music = 1167
X = list()
#c=list()
for j in xrange(total_music):
    if j+1<10:
        filename = './trainmusic/data/train/train_000'+str(j+1)+'.mp3'
    elif j+1<100:
        filename = './trainmusic/data/train/train_00'+str(j+1)+'.mp3'
    elif j+1<1000:
        filename = './trainmusic/data/train/train_0'+str(j+1)+'.mp3'
    else:
        filename = './trainmusic/data/train/train_'+str(j+1)+'.mp3'
    raw_string=''
    with audioread.audio_open(filename) as f:
        #print(f.channels, f.samplerate, f.duration)
        sr = f.samplerate
        for buf in f:
            raw_string+=buf
    raw_amp = np.fromstring(raw_string,dtype="int16")
    #print len(raw_amp),type(raw_amp)
    leftamp = raw_amp[::2]
    rightamp= raw_amp[1::2]
    #print len(leftamp),type(leftamp),len(rightamp),type(rightamp)
    zerocrossl = list()
    zerocrossr = list()
    rootmsl = list()
    rootmsr = list()
    #for i in xrange(0,len(leftamp)-winsize+1,winsize/2):
        #zerocrossl.append(zero_crossing_rate(leftamp[i:i+winsize],winsize,sr))
        #rootmsl.append(root_mean_square(leftamp[i:i+winsize],winsize,sr))
        #zerocrossr.append(zero_crossing_rate(rightamp[i:i+winsize],winsize,sr))
        #rootmsr.append(root_mean_square(rightamp[i:i+winsize],winsize,sr))
    zerocrossl=zero_crossing_rate(leftamp,winsize,sr)
    rootmsl=root_mean_square(leftamp,winsize,sr)
    zerocrossr=zero_crossing_rate(rightamp,winsize,sr)
    rootmsr=root_mean_square(rightamp,winsize,sr)
    zerocrossl=np.asarray(zerocrossl)
    rootmsl = np.asarray(rootmsl)
    zerocrossr=np.asarray(zerocrossr)
    rootmsr = np.asarray(rootmsr)
    #zerocross = zero_crossing_rate(raw_amp,winsize)
    #rootms = root_mean_square(raw_amp,winsize)
    diffzerol = np.diff(zerocrossl)
    diffzeror = np.diff(zerocrossr)
    #print len(zerocross),len(rootms)
    vector = [zerocrossl.mean(),zerocrossl.std(),zerocrossr.mean(),zerocrossr.std(),diffzerol.mean(),diffzerol.std(),diffzeror.mean(),diffzeror.std(),rootmsl.mean(),rootmsl.std(),rootmsr.mean(),rootmsr.std()]
    X.append(vector)
    if j%100==0:
        print 'current progress is '+str(j)

current progress is 0
current progress is 100
current progress is 200
current progress is 300
current progress is 400
current progress is 500
current progress is 600
current progress is 700
current progress is 800
current progress is 900
current progress is 1000
current progress is 1100


In [188]:
print vector
print len(X), type(X)
print X[0]
for i in xrange(len(X)):
    if X[i][8] == None:
        print '?'

[0.036910621642926543, 0.024794724535241529, 0.037373508221965571, 0.024761325978969018, -0.00076275973184191954, 0.028724772956820993, -0.00067942331197451725, 0.029567666475432464, 123.51168752620327, 303.62575044206756, 99.908266149817493, 270.06765843651488]
1167 <type 'list'>
[0.064186557040459044, 0.023622476205891921, 0.06508483198592091, 0.019977175275142409, -0.00015625578725137978, 0.020502584810756785, -2.1991555242786885e-05, 0.015751605744672856, 215.40536460087378, 355.68239208774287, 146.00534031629519, 313.65808964626245]


In [189]:
import sklearn as sk
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

labelcsv = pd.read_csv('music_train_labels.csv')
label = labelcsv['genre']
le = LabelEncoder()
le.fit(label.unique())
trans_label = le.transform(label)

In [217]:
rfc = RandomForestClassifier(n_estimators=50)
#sv = SVC()
rfc.fit(X[:800],trans_label[:800])
print rfc.score(X[800:1100],trans_label[800:1100])
#sv.fit(X,trans_label)
#sv.score(X,trans_label)
#dtc = DecisionTreeClassifier()
#dtc.fit(X[:800],trans_label[:800])
#print dtc.score(X[800:1100],trans_label[800:1100])
rfc.fit(X,trans_label)

0.506666666667


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [218]:
total_music_test=145
sol = dict()
sol2=dict()
for j in xrange(total_music_test):
    if j+1<10:
        fl = 'fe_test_000'+str(j+1)+'.mp3'
        filename = './testmusic/data/feature_extraction_test/fe_test_000'+str(j+1)+'.mp3'
    elif j+1<100:
        fl = 'fe_test_00'+str(j+1)+'.mp3'
        filename = './testmusic/data/feature_extraction_test/fe_test_00'+str(j+1)+'.mp3'
    elif j+1<1000:
        fl = 'fe_test_0'+str(j+1)+'.mp3'
        filename = './testmusic/data/feature_extraction_test/fe_test_0'+str(j+1)+'.mp3'
    else:
        fl = 'fe_test_'+str(j+1)+'.mp3'
        filename = './testmusic/data/feature_extraction_test/fe_test_'+str(j+1)+'.mp3'
    raw_string=''
    with audioread.audio_open(filename) as f:
        #print(f.channels, f.samplerate, f.duration)
        sr = f.samplerate
        for buf in f:
            raw_string+=buf
    raw_amp = np.fromstring(raw_string,dtype="int16")
    #print len(raw_amp),type(raw_amp)
    leftamp = raw_amp[::2]
    rightamp= raw_amp[1::2]
    #print len(leftamp),type(leftamp),len(rightamp),type(rightamp)
    zerocrossl = list()
    zerocrossr = list()
    rootmsl = list()
    rootmsr = list()
    #for i in xrange(0,len(leftamp)-winsize+1,winsize/2):
        #zerocrossl.append(zero_crossing_rate(leftamp[i:i+winsize],winsize,sr))
        #rootmsl.append(root_mean_square(leftamp[i:i+winsize],winsize,sr))
        #zerocrossr.append(zero_crossing_rate(rightamp[i:i+winsize],winsize,sr))
        #rootmsr.append(root_mean_square(rightamp[i:i+winsize],winsize,sr))
    zerocrossl=zero_crossing_rate(leftamp,winsize,sr)
    rootmsl=root_mean_square(leftamp,winsize,sr)
    zerocrossr=zero_crossing_rate(rightamp,winsize,sr)
    rootmsr=root_mean_square(rightamp,winsize,sr)
    zerocrossl=np.asarray(zerocrossl)
    rootmsl = np.asarray(rootmsl)
    zerocrossr=np.asarray(zerocrossr)
    rootmsr = np.asarray(rootmsr)
    #zerocross = zero_crossing_rate(raw_amp,winsize)
    #rootms = root_mean_square(raw_amp,winsize)
    diffzerol = np.diff(zerocrossl)
    diffzeror = np.diff(zerocrossr)
    #print len(zerocross),len(rootms)
    vector = [zerocrossl.mean(),zerocrossl.std(),zerocrossr.mean(),zerocrossr.std(),diffzerol.mean(),diffzerol.std(),diffzeror.mean(),diffzeror.std(),rootmsl.mean(),rootmsl.std(),rootmsr.mean(),rootmsr.std()]

    result = rfc.predict([vector])
    #result_sv = sv.predict(vector)
    genre = le.inverse_transform(result)
    #genresv = le.inverse_transform(result_sv)
    #print len(genre)
    sol[fl]=genre[0]
    #sol2[fl]=genresv[0]
    if j%100==0:
        print 'current progress is '+str(j)

current progress is 0
current progress is 100


In [219]:
print sol
import dill
with open ('q1solution.data','wb') as f:
    dill.dump(sol,f)

{'fe_test_0041.mp3': 'electronic', 'fe_test_0023.mp3': 'folkcountry', 'fe_test_0079.mp3': 'raphiphop', 'fe_test_0144.mp3': 'raphiphop', 'fe_test_0133.mp3': 'raphiphop', 'fe_test_0019.mp3': 'jazz', 'fe_test_0052.mp3': 'jazz', 'fe_test_0059.mp3': 'raphiphop', 'fe_test_0017.mp3': 'raphiphop', 'fe_test_0103.mp3': 'rock', 'fe_test_0111.mp3': 'folkcountry', 'fe_test_0024.mp3': 'folkcountry', 'fe_test_0012.mp3': 'folkcountry', 'fe_test_0075.mp3': 'rock', 'fe_test_0115.mp3': 'jazz', 'fe_test_0062.mp3': 'jazz', 'fe_test_0137.mp3': 'rock', 'fe_test_0055.mp3': 'jazz', 'fe_test_0093.mp3': 'rock', 'fe_test_0049.mp3': 'jazz', 'fe_test_0016.mp3': 'rock', 'fe_test_0026.mp3': 'jazz', 'fe_test_0002.mp3': 'rock', 'fe_test_0065.mp3': 'folkcountry', 'fe_test_0129.mp3': 'folkcountry', 'fe_test_0045.mp3': 'rock', 'fe_test_0057.mp3': 'rock', 'fe_test_0127.mp3': 'raphiphop', 'fe_test_0015.mp3': 'rock', 'fe_test_0124.mp3': 'raphiphop', 'fe_test_0114.mp3': 'rock', 'fe_test_0145.mp3': 'raphiphop', 'fe_test_0120.m

In [ ]:
print sol['f']

In [115]:
print sol2

{'fe_test_0041.mp3': 'rock', 'fe_test_0023.mp3': 'rock', 'fe_test_0079.mp3': 'rock', 'fe_test_0144.mp3': 'rock', 'fe_test_0133.mp3': 'rock', 'fe_test_0019.mp3': 'rock', 'fe_test_0052.mp3': 'rock', 'fe_test_0059.mp3': 'rock', 'fe_test_0017.mp3': 'rock', 'fe_test_0103.mp3': 'rock', 'fe_test_0111.mp3': 'rock', 'fe_test_0024.mp3': 'rock', 'fe_test_0012.mp3': 'rock', 'fe_test_0075.mp3': 'rock', 'fe_test_0115.mp3': 'rock', 'fe_test_0062.mp3': 'rock', 'fe_test_0137.mp3': 'rock', 'fe_test_0055.mp3': 'rock', 'fe_test_0093.mp3': 'rock', 'fe_test_0049.mp3': 'rock', 'fe_test_0016.mp3': 'rock', 'fe_test_0026.mp3': 'rock', 'fe_test_0002.mp3': 'rock', 'fe_test_0065.mp3': 'rock', 'fe_test_0129.mp3': 'rock', 'fe_test_0045.mp3': 'rock', 'fe_test_0057.mp3': 'rock', 'fe_test_0127.mp3': 'rock', 'fe_test_0015.mp3': 'rock', 'fe_test_0124.mp3': 'rock', 'fe_test_0114.mp3': 'rock', 'fe_test_0145.mp3': 'rock', 'fe_test_0120.mp3': 'rock', 'fe_test_0089.mp3': 'rock', 'fe_test_0108.mp3': 'rock', 'fe_test_0142.mp3':

In [51]:
zerocross = list()
for i in xrange(len(raw_amp)-winsize+1):    
    zerocross.append(audioop.cross(raw_amp[i:i+winsize],2))

In [52]:
#print len(zerocross)
#print np.asarray(zerocross).mean()

882631
1058.51421036


In [53]:
#print raw_amp[:1000]

In [195]:
a={'fe_test_0041.mp3': 'folkcountry', 'fe_test_0023.mp3': 'rock', 'fe_test_0079.mp3': 'raphiphop', 'fe_test_0144.mp3': 'raphiphop', 'fe_test_0133.mp3': 'raphiphop', 'fe_test_0019.mp3': 'jazz', 'fe_test_0052.mp3': 'jazz', 'fe_test_0059.mp3': 'raphiphop', 'fe_test_0017.mp3': 'raphiphop', 'fe_test_0103.mp3': 'rock', 'fe_test_0111.mp3': 'folkcountry', 'fe_test_0024.mp3': 'folkcountry', 'fe_test_0012.mp3': 'folkcountry', 'fe_test_0075.mp3': 'rock', 'fe_test_0115.mp3': 'jazz', 'fe_test_0062.mp3': 'folkcountry', 'fe_test_0137.mp3': 'jazz', 'fe_test_0055.mp3': 'folkcountry', 'fe_test_0093.mp3': 'jazz', 'fe_test_0049.mp3': 'jazz', 'fe_test_0016.mp3': 'rock', 'fe_test_0026.mp3': 'rock', 'fe_test_0002.mp3': 'rock', 'fe_test_0065.mp3': 'raphiphop', 'fe_test_0129.mp3': 'jazz', 'fe_test_0045.mp3': 'jazz', 'fe_test_0057.mp3': 'rock', 'fe_test_0127.mp3': 'raphiphop', 'fe_test_0015.mp3': 'rock', 'fe_test_0124.mp3': 'raphiphop', 'fe_test_0114.mp3': 'rock', 'fe_test_0145.mp3': 'raphiphop', 'fe_test_0120.mp3': 'raphiphop', 'fe_test_0089.mp3': 'raphiphop', 'fe_test_0108.mp3': 'rock', 'fe_test_0142.mp3': 'jazz', 'fe_test_0047.mp3': 'jazz', 'fe_test_0048.mp3': 'folkcountry', 'fe_test_0009.mp3': 'rock', 'fe_test_0116.mp3': 'rock', 'fe_test_0077.mp3': 'jazz', 'fe_test_0001.mp3': 'jazz', 'fe_test_0073.mp3': 'raphiphop', 'fe_test_0040.mp3': 'rock', 'fe_test_0058.mp3': 'raphiphop', 'fe_test_0109.mp3': 'jazz', 'fe_test_0125.mp3': 'rock', 'fe_test_0084.mp3': 'rock', 'fe_test_0032.mp3': 'folkcountry', 'fe_test_0078.mp3': 'rock', 'fe_test_0036.mp3': 'rock', 'fe_test_0135.mp3': 'folkcountry', 'fe_test_0018.mp3': 'raphiphop', 'fe_test_0056.mp3': 'jazz', 'fe_test_0098.mp3': 'rock', 'fe_test_0139.mp3': 'folkcountry', 'fe_test_0083.mp3': 'raphiphop', 'fe_test_0087.mp3': 'rock', 'fe_test_0064.mp3': 'raphiphop', 'fe_test_0074.mp3': 'rock', 'fe_test_0143.mp3': 'jazz', 'fe_test_0022.mp3': 'jazz', 'fe_test_0021.mp3': 'jazz', 'fe_test_0037.mp3': 'rock', 'fe_test_0104.mp3': 'jazz', 'fe_test_0025.mp3': 'folkcountry', 'fe_test_0134.mp3': 'jazz', 'fe_test_0140.mp3': 'rock', 'fe_test_0131.mp3': 'jazz', 'fe_test_0007.mp3': 'rock', 'fe_test_0130.mp3': 'jazz', 'fe_test_0043.mp3': 'raphiphop', 'fe_test_0092.mp3': 'jazz', 'fe_test_0082.mp3': 'rock', 'fe_test_0100.mp3': 'jazz', 'fe_test_0035.mp3': 'rock', 'fe_test_0086.mp3': 'rock', 'fe_test_0118.mp3': 'folkcountry', 'fe_test_0067.mp3': 'jazz', 'fe_test_0076.mp3': 'folkcountry', 'fe_test_0107.mp3': 'jazz', 'fe_test_0095.mp3': 'rock', 'fe_test_0096.mp3': 'rock', 'fe_test_0091.mp3': 'raphiphop', 'fe_test_0112.mp3': 'rock', 'fe_test_0020.mp3': 'rock', 'fe_test_0014.mp3': 'rock', 'fe_test_0068.mp3': 'rock', 'fe_test_0117.mp3': 'jazz', 'fe_test_0039.mp3': 'raphiphop', 'fe_test_0010.mp3': 'raphiphop', 'fe_test_0141.mp3': 'raphiphop', 'fe_test_0044.mp3': 'jazz', 'fe_test_0072.mp3': 'rock', 'fe_test_0050.mp3': 'jazz', 'fe_test_0099.mp3': 'jazz', 'fe_test_0097.mp3': 'jazz', 'fe_test_0090.mp3': 'rock', 'fe_test_0113.mp3': 'rock', 'fe_test_0105.mp3': 'jazz', 'fe_test_0088.mp3': 'rock', 'fe_test_0029.mp3': 'folkcountry', 'fe_test_0006.mp3': 'rock', 'fe_test_0008.mp3': 'rock', 'fe_test_0110.mp3': 'rock', 'fe_test_0081.mp3': 'jazz', 'fe_test_0011.mp3': 'jazz', 'fe_test_0122.mp3': 'rock', 'fe_test_0066.mp3': 'rock', 'fe_test_0005.mp3': 'rock', 'fe_test_0061.mp3': 'raphiphop', 'fe_test_0051.mp3': 'jazz', 'fe_test_0042.mp3': 'rock', 'fe_test_0126.mp3': 'jazz', 'fe_test_0013.mp3': 'rock', 'fe_test_0132.mp3': 'rock', 'fe_test_0119.mp3': 'rock', 'fe_test_0136.mp3': 'rock', 'fe_test_0094.mp3': 'rock', 'fe_test_0031.mp3': 'rock', 'fe_test_0106.mp3': 'rock', 'fe_test_0121.mp3': 'electronic', 'fe_test_0034.mp3': 'jazz', 'fe_test_0004.mp3': 'rock', 'fe_test_0070.mp3': 'jazz', 'fe_test_0046.mp3': 'folkcountry', 'fe_test_0054.mp3': 'folkcountry', 'fe_test_0080.mp3': 'jazz', 'fe_test_0030.mp3': 'rock', 'fe_test_0123.mp3': 'rock', 'fe_test_0085.mp3': 'raphiphop', 'fe_test_0028.mp3': 'raphiphop', 'fe_test_0038.mp3': 'folkcountry', 'fe_test_0003.mp3': 'raphiphop', 'fe_test_0060.mp3': 'rock', 'fe_test_0101.mp3': 'jazz', 'fe_test_0033.mp3': 'rock', 'fe_test_0102.mp3': 'folkcountry', 'fe_test_0027.mp3': 'rock', 'fe_test_0069.mp3': 'jazz', 'fe_test_0071.mp3': 'raphiphop', 'fe_test_0138.mp3': 'folkcountry', 'fe_test_0063.mp3': 'folkcountry', 'fe_test_0053.mp3': 'jazz', 'fe_test_0128.mp3': 'rock'}


In [196]:
print type(a)

<type 'dict'>
